In [27]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

def close_popup(driver):
    try:
        # Find the navigation menu button and click it to close the popup using JavaScript
        nav_menu_button = driver.find_element(By.CSS_SELECTOR, '.contentMenu__contentMenuStyles__hamburgerButton')
        driver.execute_script("arguments[0].click();", nav_menu_button)

        # Wait for the popup to be fully removed
        WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.CSS_SELECTOR, '.contentMenu__contentMenuStyles__menuContent')))
        print('Popup removed successfully.')
    except:
        print('Could not remove popup.')

def scroll_to_end(driver):
    # Scroll to the end of the page to load more job listings
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

def get_jobs(keyword, num_jobs, verbose, path, slp_time):
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''

    # Initializing the webdriver service
    service = Service(path)

    # Initializing the webdriver
    options = webdriver.ChromeOptions()

    # Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    # options.add_argument('headless')

    driver = webdriver.Chrome(service=service, options=options)
    driver.set_window_size(1120, 1000)

    url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+"&locT=&locId=&jobType="
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  # If true, should be still looking for new jobs.
        # Let the page load. Change this number based on your internet speed.
        # Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(slp_time)

        close_popup(driver)

        # Scroll multiple times to load more jobs on the page
        for _ in range(5):
            scroll_to_end(driver)
            time.sleep(2)  # Add a small delay after each scroll

        # Wait for new elements to load
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "jl")))
        except:
            print("No new job listings loaded within the timeout.")
            break

        job_buttons = driver.find_elements(By.CLASS_NAME, "jl")  # jl for Job Listing. These are the buttons we're going to click.

        for job_button in job_buttons:
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            try:
                job_button.click()
                time.sleep(1)
                collected_successfully = False
            except:
                print("Could not click on job element. Skipping.")
                continue

            while not collected_successfully:
                try:
                    company_name = driver.find_element(By.XPATH, './/div[@class="employerName"]').text
                    location = driver.find_element(By.XPATH, './/div[@class="location"]').text
                    job_title = driver.find_element(By.XPATH, './/div[contains(@class, "title")]').text
                    job_description = driver.find_element(By.XPATH, './/div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)Q

            try:
                salary_estimate = driver.find_element(By.XPATH, './/span[@class="gray salary"]').text
            except:
                salary_estimate = -1  # You need to set a "not found value. It's important."

            try:
                rating = driver.find_element(By.XPATH, './/span[@class="rating"]').text
            except:
                rating = -1  # You need to set a "not found value. It's important."

            # Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            # Going to the Company tab...
            # clicking on this:
            # <div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element(By.XPATH, './/div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    # <div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    # </div>
                    headquarters = driver.find_element(By.XPATH, './/div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except:
                    headquarters = -1

                # Rest of the code remains unchanged

            except:  # Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                # Rest of the code remains unchanged

            jobs.append({"Job Title": job_title,
                         "Salary Estimate": salary_estimate,
                         "Job Description": job_description,
                         "Rating": rating,
                         "Company Name": company_name,
                         "Location": location,
                         "Headquarters": headquarters,
                         # Rest of the code remains unchanged
                         })

        if len(jobs) >= num_jobs:
            break

    driver.quit()
    return pd.DataFrame(jobs)

path = "C:\\Users\\ASUS\\Documents\\ds_salary_proj\\chromedriver.exe"

df = get_jobs('Data Scientist', 1500, False, path, 10)
df.to_csv('Uncleaned_DS_jobs.csv', index=False)


Popup removed successfully.
No new job listings loaded within the timeout.
